In [160]:
import os

import xarray as xr
import numpy as np
import torch
from benchmark.bm.score import compute_weighted_rmse, compute_weighted_mae, compute_weighted_acc

import matplotlib
import matplotlib.pyplot as plt

import copy
import matplotlib.gridspec as gridspec

from WD.plotting import plot_map, add_label_to_axes

import cartopy.crs as ccrs
from datetime import datetime

In [161]:
names = ["Constant", "ReduceLROnPlateau", "StepLR", "CosineAnnealingLR", "CosineAnnealingWarmRestarts", "CosineAnnealingWarmupRestarts"]
ds_id = "A7B509"

In [12]:
for name in names:
    os.system("sbatch submit_script_5_lr_schedule_selection.sh -d {} -l {}".format(ds_id, name))

Submitted batch job 41759167
Submitted batch job 41759168
Submitted batch job 41759169
Submitted batch job 41759170
Submitted batch job 41759171
Submitted batch job 41759172


In [162]:
model_ids = ["23CD86","04EA1C", "8DFE77", "7AB0CF", "2DB6A9", "4206A6"]
model_ids_mse_loss = ["8F3861","0F3525", "D359F7", "2464FD", "30604C", "5C39F8"]

In [ ]:
for model_id in model_ids_mse_loss:
    os.system("sbatch submit_script_3_eval.sh -d {} -m {} -e {}".format(ds_id, model_id, 1))

In [171]:
for model_id in model_ids:
    os.system("sbatch submit_script_3_eval.sh -d {} -m {} -e {}".format(ds_id, model_id, 1))

Submitted batch job 42711086
Submitted batch job 42711087
Submitted batch job 42711088
Submitted batch job 42711089
Submitted batch job 42711090
Submitted batch job 42711091


In [173]:
predictions_experiments = {}
targets_experiments = {}


for model_id in model_ids:
    predictions_experiments[model_id] = xr.load_dataset(f"/data/compoundx/WeatherDiff/model_output/{ds_id}/{model_id}_gen.nc")
    targets_experiments[model_id] = xr.load_dataset(f"/data/compoundx/WeatherDiff/model_output/{ds_id}/{model_id}_target.nc")

In [175]:
rmse_maps = {}
rmse = {}
r2 = {}
r2_maps = {}

for model_id in model_ids:
    rmse[model_id] = compute_weighted_rmse(predictions_experiments[model_id].isel({"ensemble_member":0}), targets_experiments[model_id].isel({"ensemble_member":0}))
    rmse_maps[model_id] = compute_weighted_rmse(predictions_experiments[model_id].isel({"ensemble_member":0}), targets_experiments[model_id].isel({"ensemble_member":0}), mean_dims=("init_time", "lead_time"))
    print("{}, RMSE is {:.1f}".format(model_id, rmse[model_id].z_500.values))

23CD86, RMSE is 942.4
04EA1C, RMSE is 627.5
8DFE77, RMSE is 626.4
7AB0CF, RMSE is 633.0
2DB6A9, RMSE is 603.9
4206A6, RMSE is 620.3


In [176]:
predictions_experiments = {}
targets_experiments = {}


for model_id in model_ids_mse_loss:
    predictions_experiments[model_id] = xr.load_dataset(f"/data/compoundx/WeatherDiff/model_output/{ds_id}/{model_id}_gen.nc")
    targets_experiments[model_id] = xr.load_dataset(f"/data/compoundx/WeatherDiff/model_output/{ds_id}/{model_id}_target.nc")

In [177]:
rmse_maps = {}
rmse = {}
r2 = {}
r2_maps = {}

for model_id in model_ids_mse_loss:
    rmse[model_id] = compute_weighted_rmse(predictions_experiments[model_id].isel({"ensemble_member":0}), targets_experiments[model_id].isel({"ensemble_member":0}))
    rmse_maps[model_id] = compute_weighted_rmse(predictions_experiments[model_id].isel({"ensemble_member":0}), targets_experiments[model_id].isel({"ensemble_member":0}), mean_dims=("init_time", "lead_time"))
    print("{}, RMSE is {:.1f}".format(model_id, rmse[model_id].z_500.values))

8F3861, RMSE is 624.5
0F3525, RMSE is 638.9
D359F7, RMSE is 641.6
2464FD, RMSE is 643.3
30604C, RMSE is 622.6
5C39F8, RMSE is 645.1
